# Imbalanced Classes, XGBoost, and Imputation - Guided Task

In this assignment, we will learn about:

* XGBoosted trees, a powerful extension of the random forest model
* Resampling strategies for imbalanced data
* Imputation

## Class Imbalances

Ideally, the target classes are evenly distributed so that our model has adequate data to learn the relationship between the input variables and the target classes. For example, if you were trying to predict balh blah blah, out of 100 samples, the ideal case would be for 50 of the samples to correspond to a boy and 50 to a girl. 

However, many real-world datasets are not like this. For example, credit card fraud detection - there are billions of credit card transactions conducted each day, of which a minute percentage are fraudulent. Thus, determining whether or not a transacation is fraudulent is an example of an imbalanced class problem, as shown in this [data set](https://www.kaggle.com/mlg-ulb/creditcardfraud): of the ~285,000 samples in this data set, only 1000 of them are labeled fraudulent. Another example of this is the [Porta Seguro Safe Driving dataset](https://www.kaggle.com/c/porto-seguro-safe-driver-prediction/data?select=train.csv), which contains data about different drivers and whether or not they've filed an insurance claim. In this dataset, of the ~595,000 samples, only ~22,000 reported claims.

In a previous assignment, we talked about how to identify an imbalanced class problem and why ROC-AUC is a better metric for measuring performance in these cases. In some cases, though, you might not be able to get adequate results without addressing the class imbalance. Today, we will learn a couple different strategies for doing so.

Let's go ahead and read in some data:

This is an abbreviated dataset from a Harmful Algal Bloom prediction project done in coordination with the Iowa Department of Natural resources. Samples were taken from 38 lakes across 3 years and several chemical and biological measurements were taken. The goal of the project is to use this data to attempt to forecast whether or not a harmful algal bloom will be in the lake in the next week.

You can read more about the project [here](https://docs.google.com/presentation/d/1he1N788kiU3Bbtic2OqpTNBty6BVyAYWB0avPAEmwa0/edit?usp=sharing).

The variables are:

* microcystin - The amount of [microcystin](https://en.wikipedia.org/wiki/Microcystin) toxins measured in the sample. Microcystin is a toxin produced by [cyanobacteria](https://en.wikipedia.org/wiki/Cyanobacteria), a broad clas of bacteria that can be found in lakes.  
* x16s - Gene copies of [16s rRNA](https://en.wikipedia.org/wiki/16S_ribosomal_RNA) found in the sample
* mcy_a_m - Gene copies of the microcystinA _Microcystin_ functional gene, a gene involved in the production of microcystin
* doc_ppm - Dissolved organic carbon
* ortho_p_mg_p_l - Orthophosphate
* tkn_mg_n_l - [Total Kjeldahl nitrogen](https://en.wikipedia.org/wiki/Kjeldahl_method#Total_Kjeldahl_nitrogen), the sum of nitrogen found in organic substances, ammonia, and ammonium.
* cl_mg_cl_l - The amount of chlorine found in the sample
* p_h - The pH level of the sample.
* tp - The total amount of phosphate measured in the sample
* mcya_16s - The ratio of microcystin to 16s gene copies
* HAB_next_week - Whether or not there was a harmful algal bloom observed in the following week. A 0 indicataes there wasn't and a 1 indicates there was.

We already know basic EDA stuff from many previous examples, so this assignment will forgo most of repeating EDA and focus on class imbalances. 

Let's see how the classes break down:

So we see that there is a heavy class imbalance in the samples - there are no harmful algal blooms in over 90% of the samples, which is definitely something we need to account for in our analysis. 

Before we move on to splitting the data and training our models, let's also look at the missing values:

I've already did a lot of the cleaning for this dataset and took care of most of the missing values. However, there are still 58 `p_h` measurements missing. Since we're concerned with class imbalances, we should take a look to see how these missing values are spread across both classes:

The vast majority of the missing observations belong to the negative class at a ratio of 57:1. This is good news: the ratio of missing values is not worse than the observed ratio of missing values in the dataset at large. For comparison, it would be bad news if the ratio of missing negative class samples to positive class samples were smaller than the observed ratio - this would mean that we would have to throw away a lot more positive class samples, exacerbating the class imbalance we saw in the whole dataset.

For now, we're just going to drop all of the missing values. After we make our first set of predictions, we'll come back and learn some imputation strategies.

Let's make our testing and training sets, stratifying on our target variable:

And let's verify that the stratification worked:

In [1]:
def get_class_props(target_series):
    class_count = target_series.value_counts().rename({0: "No HAB", 1: "HAB"})
    ratio = round(class_count[0] / class_count[1], 2)
    return '\t'.join([str(x) for x in [class_count[0], class_count[1], ratio]])

We see that the % of HABs observed in each of the datasets is roughly the same. Since there are so few HAB observations, this is the best split we're going to get of each of the classes.

Okay, now that we have our data, let's go ahead and make our model.

### Homework questions 1

1. Why are class imbalances bad?
2. Why is accuracy a poor metric for prediction cases where there is a heavy class imbalance?
3. Stratifying on the target class is always important. Why is it especially important for cases with a heavy class imbalance? 
    * Hint: What's the worst-case scenario for the training-test split on an imbalanced data set?

## XGBoost

First introduced in 2016, [XGBoost](https://arxiv.org/abs/1603.02754) is a powerful prediction algorithm that works for both classification and regression. XGBoost is probably the most popular machine learning algorithm today for a few reasons:

* It's fast 
* It's easy to use
* Excellent predictions right out of the box

Much like random forests are an improvement on decision trees, XGBoost trees are an improvement on random forests. The key algorithmic improvement over random forests is in how the ensemble of weak learners is assembled. Recall that a random forest is a collection of a lot of decision trees that are made from randomly sampling the features at each decision stump. There is no particular method to assembling these random trees. 

On the other hand, XGBoost grows the collection of trees one at a time by adding a tree that will decrease the loss function at each step. In other words, when adding the *n*th tree, the tree is constructed in such a way as to decrease the overall (say) ROC AUC. The weight (learning rate, or eta) of this given tree is based on the gradient of the loss function and the previous model's output. This is where the name XGBoost comes from: XGBoost is short for eXtreme Gradient Boosted trees.

Besides this main difference, here are other ways in which XGBoost improves upon random forests.

* Model improvements: Uses gradient boosting to optimize the prediction of the trees and implements L1 and L2 regularization (how LASSO and ridge regression prevent overfitting) into its feature weighting
* Algorithmic improvements: XGBoost handles sparse matrices really well (which is a problem in high dimensional data), natively handles missing data, and support continued training so that you can improve an existing XGBoosted tree when you get new data
* Computational improvements: XGBoost is highly parallelized and takes advantage of system archictecture and caches results for future calculations in a clever way. This means that it is __fast__ and supports incremental learning.

To learn more about it, see these videos:

* [Visual Guide to Gradient Boosted Trees (xgboost)](https://www.youtube.com/watch?v=TyvYZ26alZs)
* [XGBoost: How it works, with an example](https://www.youtube.com/watch?v=OQKQHNCVf5k)

And read the [original paper from Tianqi Chen](https://arxiv.org/abs/1603.02754).

To implement the XGBoosted trees, we're going to use the `xgboost` package. You can install is using `pip install xgboost`. After doing so, you'll be able to follow along below.

In [2]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, plot_confusion_matrix

The API for using XGBoost should be quite familiar to you since you've been using `sklearn` all semester. Just like those models, we need to instantiate it and fit it to our training set.

In [3]:
def get_roc_auc_score(classifier, X_test, y_test):
    y_pred = classifier.predict_proba(X_test)[:, 1]
    return roc_auc_score(y_test, y_pred)
 

While our ROC-AUC was in the high 80% range, there is still room for improvement. Our first improvement will be in addressin the class imbalance in the dataset.

### Homework Questions 2

1. What does a confusion matrix tell us?
2. What would a perfect confusion matrix look like?
3. What is concerning about the above confusion matrix?

## The SMOTE Algorithm

Techniques to handle class imbalances are an active area of study. Two broad classes of approaches to correcting class imbalances are _oversampling_ and _undersampling_.

* __Oversampling__ seeks to correct a class imbalance by repeatedly sampling the minority class, either by taking repeated samples or by simulating minority class samples.
* __Undersampling__ approaches the problem by using fewer samples from the majority class. For instance, if there were 1000 samples of Class A and 200 of Class B, then an undersampling-based approach could be to randomly choose only 200 samples from Class A.

The one we're going to use today is the SMOTE algorithm, which stands for _synthetic minority over-sampling technique_. This is an oversampling technique based on simulating the minority classes. The steps of the SMOTE algorithm are:

* Randomly pick a member _x_ of the minority class
* Find its _k_ nearest neighbors
* Of those _k_ neighbors, choose one at random (let's call it _x'_)
* Create a new point ont the line between _x_ and _x'_
* Repeat until the classes are even



### Homework Questions 3

1. What are some cases where undersampling is preferable to oversampling, and vice versa? 
2. Implement an Undersampling technique from the [`imbalanced-learn` library](https://imbalanced-learn.org/stable/user_guide.html#user-guide) and compare the results with those obtained from the SMOTE algorithm. 

## Imputation

Now that we've implemented an XGBoost model and have done some oversampling, let's return to the missing values we threw away earlier. Even though it's always a bummer to throw away data, this current situation isn't too bad because of the class imbalance as we discussed. However, many real datasets aren't as neat as this one was and have huge amounts of missing data. 

In these cases, it is desirable to impute the data that is missing so that we can 

Imputation is the process of calculating missing values. There are [many different imputation techniques](https://scikit-learn.org/stable/modules/impute.html) - some simple methods are to fill in the missing values with the median or mean value of the column. For this dataset, we're going to impute missing values via *k*-nearest neighbors. For each missing value, we'll fill in the missing value with the mean of the *k* samples closest to that point. Let's see how it works.

First, we need to go back to our original dataset, before we dropped the missing values, and work from there.

Next, we'll load the `KNNImputer` from `sklearn.impute` and create an imputer instance:

Just like when training models, we need to first `fit` the imputer to the training dataset, then use that `imputer` to transform the training set. After it is fit on training set, we can use the imputer to impute the missing values on the testing set.

From here, we can proceed as we did before

We see that our imputation slightly improved the results.

### Homework Questions 4

1. Implement another imputer from `sklearn.impute`. See the [sklearn docs](https://scikit-learn.org/stable/modules/impute.html) to learn more about imputation and to see other imputers.
2. Notice that we called `knn_impute.fit_transform` on the training set, but only called `.transform` on the testing set. Why is this important? 
    * Hint: Look up data leakage
3. Notice that while the ROC-AUC was bigger in the last version of our model, the [specificity (or true negative rate)](https://en.wikipedia.org/wiki/Sensitivity_and_specificity#Specificity) was lower. Between these two models, which one would you pick? Why?
4. Repeat this workflow on the abridged credit card fraud dataset located `../data/creditcard.csv`. This modified dataset has missing values and a class imbalance on the order of what we saw with the HAB dataset.